[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/James-Leslie/deep-collaborative-filtering/blob/master/movielens_skorch.ipynb)

# To do:
  - use cross validation in grid search (ML100K README explains how to use 5-fold CV for evaluation)
  - regularize embedding layer
  - try [this link](https://github.com/keras-team/keras/issues/9001) for making an SkLearn base estimator (look for comment by hughfdjackson)
  - create object classes for models
  - implement TF 2.0 data classes
  - [paperswithcode link](https://paperswithcode.com/sota/collaborative-filtering-on-movielens-100k)
  - [ML 100k state of the art paper](https://arxiv.org/pdf/1706.02263v2.pdf) (RMSE=0.905): details their evaluation method
  - include genre model in grid search

## Best CV score: 0.920671 (25 : 64 : 0.2)

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Add, Dense, Concatenate, Dropout, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.math import add

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

%matplotlib inline

In [2]:
# path = 'https://raw.githubusercontent.com/James-Leslie/deep-collaborative-filtering/master/data/ml-100k/'  # access from anywhere
path = 'data/ml-100k/'  # if the files are local
df = pd.read_csv(path+'ratings.csv')

In [3]:
df.head()

,userId,movieId,rating
0,0,0,3
1,1,1,3
2,2,2,1
3,3,3,2
4,4,4,1


In [4]:
df.shape

(100000, 3)

In [5]:
print('Number of users:', df.userId.nunique())
print('Number of items:', df.movieId.nunique())
print("Min item rating:", df.rating.min())
print("Max item rating:", df.rating.max())
print("Mean item rating:", df.rating.mean())

Number of users: 943
Number of items: 1682
Min item rating: 1
Max item rating: 5
Mean item rating: 3.52986


# Create baseline features
For each different train / test split, global mean and biases need to be calculated so as to avoid potential data leakage.

In [6]:
def get_baseline(df, train_index, test_index):
    train = df.iloc[train_index]
    test = df.iloc[test_index]
    
    # compute global mean
    global_mean = train.rating.mean()

    # compute average user ratings
    user_averages = train.groupby('userId') \
        .agg({'rating':'mean'}) \
        .rename({'rating': 'user_avg'}, axis=1) \
        .reset_index()
    # add as column to train and test
    train = pd.merge(train, user_averages, how='left', on='userId')
    test = pd.merge(test, user_averages, how='left', on='userId').fillna(global_mean)

    # compute average item ratings
    item_averages = train.groupby('movieId') \
        .agg({'rating':'mean'}) \
        .rename({'rating': 'item_avg'}, axis=1) \
        .reset_index()
    # add as column to train and test
    train = pd.merge(train, item_averages, how='left', on='movieId')
    test = pd.merge(test, item_averages, how='left', on='movieId').fillna(global_mean)

    train['bias'] = (train['user_avg'] + train['item_avg'])/2 - global_mean
    test['bias'] = (test['user_avg'] + test['item_avg'])/2 - global_mean
    
    return train, test, global_mean

## Ratings model

### Grid search
**To do**:
  - tune dropout rates and optimiser
  - measure impact on accuracy of genre model

In [17]:
def compile_model(n_users, n_items, mean_rating, n_factors=25, n_hidden_1=0, n_hidden_2=0, dropout_1=.1, dropout_2=.1):
    
    # TODO: n_items, n_users and mean_rating are all hard coded at the moment
    
    # item latent factors
    item_in = Input(shape=[1])  # name='item'
    item_em = Embedding(n_items, n_factors)(item_in)
    item_vec = Flatten()(item_em)
    
    # user latent factors
    user_in = Input(shape=[1])
    user_em = Embedding(n_users, n_factors)(user_in)
    user_vec = Flatten()(user_em)
    
    # user x item bias
    bias = Input(shape=[1])
    
    # if there is a hidden layer
    if n_hidden_1:
        # concatenate user and item vectors
        conc = Concatenate()([item_vec, user_vec])
        # hidden layer 1
        hidden_1 = Dense(n_hidden_1)(conc)
        leaky_1 = LeakyReLU(alpha=0.1)(hidden_1)
        drop_1 = Dropout(dropout_1)(leaky_1)
        
        # if there is a second hidden layer
        if n_hidden_2:
            # hidden layer 2
            hidden_2 = Dense(n_hidden_2, activation='leaky_relu')(drop_1)
            drop_2 = Dropout(dropout_2)(hidden_2)
            
            # unscaled output
            out = Dense(1)(drop_2)
        else:
            out = Dense(1)(drop_1)
        
    # if there are no hidden layers
    else:
        out = Dot(name="Dot-Product", axes=1)([item_vec, user_vec])
    
    rating = add(Add()([out, bias]), mean_rating)
    
    # create model and compile it
    model = Model([user_in, item_in, bias], rating)
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    return model

In [20]:
# hyper parameters
HP_N_FACTORS = [10, 20, 30, 50]
HP_N_HIDDEN = [10, 20, 40, 60]
HP_DROPOUT = [.2, .25, .3]

In [21]:
# dataframe to store results of grid search
grid_results = []
best_loss = 1
searches = 1

n_models = len(HP_N_FACTORS) * len(HP_N_HIDDEN_1) * len(HP_DROPOUT_1)

print(f'Fitting total of {n_models} models\n')

for N_FACTORS in HP_N_FACTORS:
    for N_HIDDEN in HP_N_HIDDEN:
        for DROPOUT in HP_DROPOUT:
            
            print(f'Fitting model #{searches} with {N_FACTORS}: {N_HIDDEN} architecture, {DROPOUT} dropout rate')
            searches += 1
            
            # CV split
            # calculate global mean rating and bias
            kf = KFold(n_splits=5)
            
            total_loss, count = 0, 1
            min_epochs = 10
            
            # do CV split and compute baseline predictors
            for train_index, test_index in kf.split(df):
                train, test, global_mean = get_baseline(df, train_index, test_index)
                
                # compile model with chosen h-params
                model = compile_model(
                    n_users = df.userId.nunique(),
                    n_items = df.movieId.nunique(),
                    mean_rating = global_mean,
                    n_factors = N_FACTORS,
                    n_hidden_1 = N_HIDDEN,
                    n_hidden_2 = 0,
                    dropout_1 = DROPOUT,
                    dropout_2 = 0
                )

                result = model.fit(x=[train.userId.values, train.movieId.values, train.bias.values],
                                   y=train.rating.values, 
                                   batch_size=256,
                                   epochs=10,
                                   verbose=0,
                                   validation_data=([test.userId.values, test.movieId.values, test.bias.values], test.rating.values))
                
                fold_loss = np.sqrt(np.min(result.history['val_loss']))
                total_loss += fold_loss
                min_epochs = min(np.argmin(result.history['val_loss']) + 1, min_epochs)
                
                print(f'CV split #{count}: RMSE={fold_loss:.4f}')
                count += 1

            avg_loss = total_loss / 5
            best_loss = min(avg_loss, best_loss)
            print(f'_____________________________________CV avg RMSE={avg_loss:.4f}')
            
#             plt.plot(result.history['loss'], label='train')
#             plt.plot(result.history['val_loss'], label='val')
#             plt.axhline(y=best_loss, color='r', lw=1, ls='-')
#             plt.legend()
#             plt.show()
            
            grid_results.append({'n_factors':N_FACTORS,
                                 'n_hidden':N_HIDDEN,
                                 'dropout':DROPOUT,
                                 'val_rmse':avg_loss,
                                 'val_epochs':min_epochs,
                                 'train_hist':result.history['loss'],
                                 'val_hist':result.history['val_loss']})
            print()
            break
        break
    break

grid_results = pd.DataFrame(data=grid_results, columns=grid_results[0].keys())

Fitting total of 48 models

Fitting model #1 with 25: 64 architecture, 0.2 dropout rate
CV split #1: RMSE=0.9312
CV split #2: RMSE=0.9235
CV split #3: RMSE=0.9181
CV split #4: RMSE=0.9144
CV split #5: RMSE=0.9130
_____________________________________CV avg RMSE=0.9201



In [10]:
grid_results.sort_values('val_rmse').head(20)

,n_factors,hidden_1,hidden_2,val_rmse,val_epochs,train_hist,val_hist
18,25,64,0,0.920671,6,"[0.9027691205978393, 0.8514909013748169, 0.821...","[0.8927469430923461, 0.8680867597579957, 0.855..."
29,50,64,0,0.920688,5,"[0.899303620147705, 0.8450768502235413, 0.8144...","[0.8880050428390502, 0.8602351546287537, 0.852..."
7,10,64,0,0.920844,8,"[0.9057428205490112, 0.8551031059265136, 0.829...","[0.8932568130493164, 0.8695466453552246, 0.857..."
3,10,32,0,0.920926,9,"[0.9065328149795532, 0.8605140323638916, 0.838...","[0.8930669300079346, 0.8750645198822021, 0.863..."
25,50,32,0,0.921158,5,"[0.9001519898414612, 0.8492190532684326, 0.820...","[0.8890793560028076, 0.8680045137405396, 0.853..."
26,50,32,8,0.923065,6,"[0.9113345783233643, 0.8623110752105713, 0.835...","[0.9018789421081543, 0.8749676982879638, 0.860..."
11,25,16,0,0.923167,7,"[0.9061017292022705, 0.8587554882049561, 0.835...","[0.8932652202606202, 0.8743310714721679, 0.860..."
19,25,64,8,0.923261,6,"[0.9100314079284668, 0.8642409670829773, 0.835...","[0.8987446979522705, 0.8688390436172485, 0.856..."
9,10,64,16,0.923297,7,"[0.9099940519332885, 0.8610564653396606, 0.834...","[0.8937595949172974, 0.8704151714324951, 0.858..."
22,50,16,0,0.923403,8,"[0.9018851215362549, 0.8531892742156982, 0.830...","[0.8868837799072266, 0.8679521892547607, 0.860..."
